In [7]:
import pandas as pd
import numpy as np

In [8]:
#Input File
scoring_input1 = 'Input/Summary_Input.xlsx'
scoring_input2 = 'Input/Scoring_Format.xlsx'

In [9]:
df = pd.read_excel(scoring_input1)

In [10]:
df.shape

(685, 9)

In [11]:
brand = df['Brand'].unique()

In [12]:
brand = list(('Hector', 'Gloster', 'ZSEV', 'Hector Plus', 'Tata Harrier','Hyundai Creta'))

In [13]:
result = []
for i in brand:
    #df1 = df[df['Brand']== i].reset_index() #change this for each brand
    #del df['index']
    #del df['Sno']

    topic=pd.DataFrame({'Freq_Per_Polarity' : df[df['Brand']==i].groupby(['Dimension','Theme','Topic','Manual_Polarity']).size()}).unstack().reset_index()
    topic = topic.replace(to_replace= np.nan, value=0) 
    topic['numerator'] = topic[('Freq_Per_Polarity',   1)] - topic[('Freq_Per_Polarity',  -1)]
    topic['denominator'] = topic[('Freq_Per_Polarity',   1)] + topic[('Freq_Per_Polarity',  -1)] + 1
    topic['Topic_Score'] = topic['numerator']/topic['denominator']

    fd = pd.read_excel(scoring_input2)
#fd1=fd[fd.columns[-2:]]
    di = dict(zip(fd.Topic,fd.Topic_Wtg))

    topic['Topic_Wtg'] = topic['Topic'].map(di)
    topic['Wtd_Topic_Score'] = topic['Topic_Wtg'] * topic['Topic_Score']



    topic['Topic_Norm']=(topic['Wtd_Topic_Score']-topic['Wtd_Topic_Score'].min())/(topic['Wtd_Topic_Score'].max()-topic['Wtd_Topic_Score'].min())
    topic['Topic_Final'] = topic['Topic_Norm'].apply(lambda x: x*100)
    
    theme= topic.groupby(['Dimension','Theme']).agg(Topic = ('Topic',','.join), Sum_Topic_Wtg = ('Topic_Wtg','sum'), Sum_Wtd_Topic_Score=('Wtd_Topic_Score','sum')).reset_index()
    theme['Theme_Rollup_Score'] = theme['Sum_Wtd_Topic_Score']/theme['Sum_Topic_Wtg']
    theme['Theme_Rollup_Score']=(theme['Theme_Rollup_Score']-theme['Theme_Rollup_Score'].min())/(theme['Theme_Rollup_Score'].max()-theme['Theme_Rollup_Score'].min())
    theme['Theme_Rollup_Score']=theme['Theme_Rollup_Score'].apply(lambda x: x*100)
    dimension = theme.groupby(['Dimension']).agg(Theme = ('Theme',','.join), Sum_Topic_Wtg = ('Sum_Topic_Wtg','sum'), Sum_Wtd_Topic_Score=('Sum_Wtd_Topic_Score','sum')).reset_index()   
    dimension['Dimension_Rollup_Score'] = dimension['Sum_Wtd_Topic_Score']/dimension['Sum_Topic_Wtg']
    dimension['Dimension_Rollup_Score']=(dimension['Dimension_Rollup_Score']-dimension['Dimension_Rollup_Score'].min())/(dimension['Dimension_Rollup_Score'].max()-dimension['Dimension_Rollup_Score'].min())
    dimension['Dimension_Rollup_Score']=dimension['Dimension_Rollup_Score'].apply(lambda x: x*100)
    gi = dict(zip(dimension.Dimension,dimension.Dimension_Rollup_Score))
    theme['Dimension_Rollup_Score'] = theme['Dimension'].map(gi)
    topic=pd.DataFrame({'Freq_Per_Polarity' : df[df['Brand']==i].groupby(['Campaign','Dimension','Theme','Topic','Manual_Polarity']).size()}).unstack().reset_index()
    topic = topic.replace(to_replace= np.nan, value=0)
    topic['numerator'] = topic[('Freq_Per_Polarity',   1)] - topic[('Freq_Per_Polarity',  -1)]
    topic['denominator'] = topic[('Freq_Per_Polarity',   1)] + topic[('Freq_Per_Polarity',  -1)] + 1
    topic['Topic_Score'] = topic['numerator']/topic['denominator']

    fd = pd.read_excel(scoring_input2)
#fd1=fd[fd.columns[-2:]]
    di = dict(zip(fd.Topic,fd.Topic_Wtg))

    topic['Topic_Wtg'] = topic['Topic'].map(di)
    topic['Wtd_Topic_Score'] = topic['Topic_Wtg'] * topic['Topic_Score']

    topic['Topic_Norm']=(topic['Wtd_Topic_Score']-topic['Wtd_Topic_Score'].min())/(topic['Wtd_Topic_Score'].max()-topic['Wtd_Topic_Score'].min())
    topic['Topic_Final'] = topic['Topic_Norm'].apply(lambda x: x*100)
    channel = topic.groupby(['Campaign']).agg(Topic = ('Topic',','.join), Sum_Topic_Wtg = ('Topic_Wtg','sum'), Sum_Wtd_Topic_Score=('Topic_Final','sum')).reset_index() 

    channel['Channel_Rollup_Score'] = channel['Sum_Wtd_Topic_Score']/channel['Sum_Topic_Wtg']
    channel['Channel_Rollup_Score']=(channel['Channel_Rollup_Score']-channel['Channel_Rollup_Score'].min())/(channel['Channel_Rollup_Score'].max()-channel['Channel_Rollup_Score'].min())
    channel['Channel_Rollup_Score'] =channel['Channel_Rollup_Score'].apply(lambda x: x*100)
    channel['Sum_Wtg_Brand'] = channel['Sum_Topic_Wtg'].sum()
    channel['Sum_Wtd_Topic_Brand'] = channel['Sum_Wtd_Topic_Score'].sum()
    channel['Brand_Rollup_Score'] = channel['Sum_Wtd_Topic_Brand']/channel['Sum_Wtg_Brand']
    #channel['Brand_Rollup_Score']=(channel['Brand_Rollup_Score']-channel['Brand_Rollup_Score'].min())/(channel['Brand_Rollup_Score'].max()-channel['Brand_Rollup_Score'].min())

    channel=channel.rename(columns={'Topic':'topic'})

    final = pd.concat([theme,channel], axis=1)
    del final['Sum_Topic_Wtg']
    del final['Sum_Wtd_Topic_Score']
    del final['Sum_Wtg_Brand']
    del final['Sum_Wtd_Topic_Brand']
    del final['topic']
    del final['Topic']

    final['Brand'] = i
    final = final[['Brand','Brand_Rollup_Score','Campaign','Channel_Rollup_Score','Dimension','Dimension_Rollup_Score','Theme','Theme_Rollup_Score']]
    result.append(final)
    

In [8]:
result

[     Brand  Brand_Rollup_Score               Campaign  Channel_Rollup_Score  \
 0   Hector           74.817541  Assured Paytm Voucher              3.728527   
 1   Hector           74.817541             EMI Offers            100.000000   
 2   Hector           74.817541         ItsAHumanThing             35.107169   
 3   Hector           74.817541                 Others              2.933673   
 4   Hector           74.817541            StoriesOfMG              0.000000   
 5   Hector                 NaN                    NaN                   NaN   
 6   Hector                 NaN                    NaN                   NaN   
 7   Hector                 NaN                    NaN                   NaN   
 8   Hector                 NaN                    NaN                   NaN   
 9   Hector                 NaN                    NaN                   NaN   
 10  Hector                 NaN                    NaN                   NaN   
 11  Hector                 NaN         

In [9]:
res1 = pd.DataFrame(result[0])
res2 = pd.DataFrame(result[1])
res3 = pd.DataFrame(result[2])
res4 = pd.DataFrame(result[3])
res5 = pd.DataFrame(result[4])
res6 = pd.DataFrame(result[5])

In [10]:
final = pd.concat([res1,res2,res3,res4,res5,res6]).reset_index()
del final['index']

In [11]:
final

,Brand,Brand_Rollup_Score,Campaign,Channel_Rollup_Score,Dimension,Dimension_Rollup_Score,Theme,Theme_Rollup_Score
0,Hector,74.817541,Assured Paytm Voucher,3.728527,Customer Service,0.000000,Post Sales Service,0.000000
1,Hector,74.817541,EMI Offers,100.000000,Customer Service,0.000000,Sales Service,67.058824
2,Hector,74.817541,ItsAHumanThing,35.107169,Design,9.833467,Automotive Design,66.610644
3,Hector,74.817541,Others,2.933673,Design,9.833467,Interior Design,13.725490
4,Hector,74.817541,StoriesOfMG,0.000000,Interiors,100.000000,Comfort,100.000000
...,...,...,...,...,...,...,...,...
57,Hyundai Creta,NaN,NaN,NaN,Performance,88.909294,Mileage,86.538462
58,Hyundai Creta,NaN,NaN,NaN,Performance,88.909294,Performance,73.200993
59,Hyundai Creta,NaN,NaN,NaN,Performance,88.909294,Safety,76.923077
60,Hyundai Creta,NaN,NaN,NaN,Price,100.000000,Premium Pricing,100.000000


In [13]:
final.to_excel('Brand_Scoring.xlsx')